In [12]:
# verified

import json
import io
import sys
import os

from os.path import join

sys.path.append('../..')
from squad_tools import load_dict

In [2]:
glove_ver = '6B'

glove_path = '/pio/data/data/glove_vec/6B/glove.6B.300d.txt'
# squad_preprocced_path = '/pio/data/data/squad/glove' + glove_ver
squad_preprocced_path = '/pio/data/data/squad/test'
out_path = join(squad_preprocced_path, 'squad_neg_cut')

wordlist = load_dict(glove_path)

with open(join(squad_preprocced_path, 'train.json')) as f:
    train = json.load(f)
with open(join(squad_preprocced_path, 'train_words.json')) as f:
    train_words = json.load(f)
with open(join(squad_preprocced_path, 'train_char_ascii.json')) as f:
    train_char = json.load(f)
with open(join(squad_preprocced_path, 'train_bin_feats.json')) as f:
    train_bin_feats = json.load(f)

In [3]:
def cut_sentence_with_ans(ans, context):
    i, j = min(ans), max(ans)
    while i > 0 and context[i-1] != '.':
        i -= 1
    while context[j] != '.' and j < len(context) - 1:
        j += 1
    return i, j + 1

In [4]:
NAW_tok = u'<not_a_word>'
NAW_idx = wordlist.index(NAW_tok)
NAW_chars = [1, 3, 2]

In [7]:
train_new = []
train_words_new = []
train_char_new = []
train_bin_feats_new = []

for k in range(len(train)):
    if not train_words[k][0]:
        continue
        
    ans = train_words[k][0][0]
    i, j = cut_sentence_with_ans(ans, train[k][2])
    
    new_raw = train[k][2][:i] + train[k][2][j:] + [NAW_tok]
    train_new.append([[], train[k][1], new_raw])
    
    new_words = train_words[k][2][:i] + train_words[k][2][j:] + [NAW_idx]
    new_ans = [[len(new_words) - 1]]
    train_words_new.append([new_ans, train_words[k][1], new_words])
    
    new_char = train_char[k][1][:i] + train_char[k][1][j:] + [NAW_chars]
    train_char_new.append([train_char[k][0], new_char])
    
    new_bin_feats = train_bin_feats[k][:i] + train_bin_feats[k][j:] + [False]
    train_bin_feats_new.append(new_bin_feats)

In [13]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

with open(join(out_path, 'train_words.json'), 'w') as f:
    json.dump(train_words_new, f)
    
with open(join(out_path, 'train_char_ascii.json'), 'w') as f:
    json.dump(train_char_new, f)
    
with open(join(out_path, 'train_bin_feats.json'), 'w') as f:
    json.dump(train_bin_feats_new, f)
    
with open(join(out_path, 'train.json'), 'w') as f:
    json.dump(train_new, f)